In [1]:
!pip install pyspark==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=1715465b32813954dd1ad77cccf2d6015e736428d8606b54b4b7dbb43a142bec
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!pip install findspark

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
import findspark
findspark.init()

In [5]:
# Conectando ao google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
spark = SparkSession.builder.appName("standardscaler").getOrCreate()

In [7]:
from pyspark.ml.feature import StandardScaler

In [8]:
carros = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Spark/datas/Carros.csv", header=True, inferSchema=True, sep=";")
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import VectorAssembler
vecassembler = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="Matriz")
carros_vetor = vecassembler.transform(carros)
carros_vetor.select("Consumo", "Cilindros", "Cilindradas", "Matriz").show()

+-------+---------+-----------+------------------+
|Consumo|Cilindros|Cilindradas|            Matriz|
+-------+---------+-----------+------------------+
|     21|        6|        160|  [21.0,6.0,160.0]|
|     21|        6|        160|  [21.0,6.0,160.0]|
|    228|        4|        108| [228.0,4.0,108.0]|
|    214|        6|        258| [214.0,6.0,258.0]|
|    187|        8|        360| [187.0,8.0,360.0]|
|    181|        6|        225| [181.0,6.0,225.0]|
|    143|        8|        360| [143.0,8.0,360.0]|
|    244|        4|       1467|[244.0,4.0,1467.0]|
|    228|        4|       1408|[228.0,4.0,1408.0]|
|    192|        6|       1676|[192.0,6.0,1676.0]|
|    178|        6|       1676|[178.0,6.0,1676.0]|
|    164|        8|       2758|[164.0,8.0,2758.0]|
|    173|        8|       2758|[173.0,8.0,2758.0]|
|    152|        8|       2758|[152.0,8.0,2758.0]|
|    104|        8|        472| [104.0,8.0,472.0]|
|    104|        8|        460| [104.0,8.0,460.0]|
|    147|        8|        440|

In [11]:
escala = StandardScaler(inputCol="Matriz", outputCol="standard", withStd=True, withMean=False)
modelo = escala.fit(carros_vetor)
carrosstandard = modelo.transform(carros_vetor)
carrosstandard.select("Matriz", "standard").show(truncate=False)

+------------------+-----------------------------------------------------------+
|Matriz            |standard                                                   |
+------------------+-----------------------------------------------------------+
|[21.0,6.0,160.0]  |[0.24996122082808128,3.359609874407659,0.20137542427273997]|
|[21.0,6.0,160.0]  |[0.24996122082808128,3.359609874407659,0.20137542427273997]|
|[228.0,4.0,108.0] |[2.713864683276311,2.239739916271773,0.13592841138409947]  |
|[214.0,6.0,258.0] |[2.5472238693909235,3.359609874407659,0.32471787163979315] |
|[187.0,8.0,360.0] |[2.2258451568976763,4.479479832543546,0.4530947046136649]  |
|[181.0,6.0,225.0] |[2.15442766523251,3.359609874407659,0.28318419038354053]   |
|[143.0,8.0,360.0] |[1.7021168846864583,4.479479832543546,0.4530947046136649]  |
|[244.0,4.0,1467.0]|[2.904311327716754,2.239739916271773,1.8463609213006844]   |
|[228.0,4.0,1408.0]|[2.713864683276311,2.239739916271773,1.7721037336001115]   |
|[192.0,6.0,1676.0]|[2.28535